# Merging Data 1

In [ ]:
import os
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Join Types

After we load up the basic packages lets go ahead a merge some data. First we will start with the same data set we worked with last time. 

In [ ]:
os.chdir('data')

In [ ]:
food_data = pd.read_csv('food_all.csv', encoding='latin-1')

We don't need all the columns we did last time, so lets just keep a couple for today.

In [ ]:
keep_cols = ['ccode', 'country', 'leader', 'year', 'elected', 'age', 'male',
       'militarycareer', 'government',
       'prev_conflict', 'pt_suc', 'pt_attempt', 'precip', 'couprisk',
       'democracy', 'autocracy', 'day', 'date', 'GEO',
       '_ISO3N_', 'M49 Code_x', 'percap cals']

In [ ]:
food_data = food_data[keep_cols]

In [ ]:
food_data.head()

Alright so we have a few columns that can serve as grouping variables in this data set. Let's try to get some additional data in our "data" folder. 

First go to this website: http://www.politicalterrorscale.org/Data/Download.html

Click on the download option for the csv file. Once this is done go ahead and move the file into the data folder.  You will have to copy and paste, or just drage the file from Downloads into the labs>data folder. Next we will just load in that data.

In [ ]:
pts_data = pd.read_csv('PTS-2019.csv', encoding='latin-1')

Lets check out what this data looks like now. 

In [ ]:
pts_data.tail()

![title](img/ptsscale.png)

So we have a number of columns that can potential serve as grouping units, but lets go ahead and use the one that is numerical and coincides with a column from our food data. We only need the values for the PTS scale (A = amnesty international, H= human rights, s=State department.

In [ ]:
pts_cols = ['Year', 'COW_Code_N', 'PTS_A', 'PTS_H', 'PTS_S']

In [ ]:
pts_data = pts_data[pts_cols]

Lets peak at the two year variables within the datasest.

In [ ]:
pts_data.Year.describe()

In [ ]:
food_data.year.describe()

We now know what the years should look like in our final data sets. Lets go ahead and use the grouping unit (Cow number) and the time unit(year) to merge our two data sets. We will see what happens when trying each of the join types.

In [ ]:
left_merge = pd.merge(food_data, pts_data, left_on=['year', 'ccode'], right_on=['Year','COW_Code_N'], how='left')

In [ ]:
left_merge[['Year','year']].describe()

That's odd. The food data set has a value of years larger than the original. Since this was a left join, we had to keep each observation from the left data set (food).  But in the case that the merge variables (year and cowcode) do not uniquely identify values in the right data set, we can actually get a larger data set all together. Lets find our duplicated values.

In [ ]:
left_merge[left_merge.duplicated(['year', 'ccode'], keep=False)]

Seems that the PTS data has duplicated all of the years during the Yugoslav wars for Serbia and Yugoslavia, even though those countries are not recognized as independent in the Food data set during that time period. These sorts of issues pop up when combining data, and they can have important theoretical implications. For the purposes of todays lab, lets just drop the first duplicated values in our data and see if that fixes the issues.

In [ ]:
left_merge = left_merge.drop_duplicates(subset=['year', 'ccode'], keep='last')

In [ ]:
left_merge[['Year','year']].describe()

Now it all checks out! We can see that the PTS data merges on just under 8000 of its observations with the food data set. But what if we want to keep all the PTS data?

In [ ]:
right_merge = pd.merge(food_data, pts_data, left_on=['year', 'ccode'], right_on=['Year','COW_Code_N'], how='right')

In [ ]:
right_merge[['Year','year']].describe()

Here we can see that all PTS data is kept, and only the food data that matches (just under 8000 observations) are kept. What if we only keep the stuff that matches?

In [ ]:
inner_merge = pd.merge(food_data, pts_data, left_on=['year', 'ccode'], right_on=['Year','COW_Code_N'], how='inner')

In [ ]:
inner_merge[['Year','year']].describe()

And finally lets keep everything!

In [ ]:
outer_merge = pd.merge(food_data, pts_data, left_on=['year', 'ccode'], right_on=['Year','COW_Code_N'], how='outer')

In [ ]:
outer_merge[['Year','year']].describe()

For fun lets see some descriptives while the data is loaded up. (we'll use the left join since thats the most theoretically concrete UOA)

In [ ]:
left_merge.groupby('government')[['PTS_A', 'PTS_S','PTS_H']].mean()

Not very suprising, warlordism is quite clearly the highest on the political terror scales. Parliamentary democracy furthermore is much lower than everything while presidential democracy is in general higher than monarchies.

## Merging Tabular with Spatial Data

Alright time to put some of this data to spatial work. 

In [ ]:
import geopandas as gpd
%matplotlib notebook
from shapely.geometry import Point, LineString, Polygon

In [ ]:
world = gpd.read_file('worldmap/cshapes.shp') 

In [ ]:
world.plot()

In [ ]:
world.head()

This is a shapefile which has an excellent number of grouping units as well as the proper geographical borders of entities within the international system at a given time. Found here http://nils.weidmann.ws/projects/cshapes.html

In [ ]:
world = world[world.COWEYEAR == 2016]

In [ ]:
world.plot()

Now we have a world map! Let's check out the PTS values for each country in a single year.

In [ ]:
pts_2016 = left_merge[left_merge.year == 2014]

In [ ]:
world_pts = world.merge(pts_2016, left_on='COWCODE', right_on='ccode', how='left')

With this merge we make sure that we merge on the spatial file, so that way we still have the geometry column.

In [ ]:
world_pts.head()

In [ ]:
world_pts.plot(column='PTS_A', legend=True)

## Spatial to Spatial Joins

Now we have a data cross-sectional spatial data frame of 2014, lets do some joins to some points.

In [ ]:
ged_2014 = pd.read_csv('ged_2014.csv')

In [ ]:
ged_2014.head()

We are working with the UCDP GED data set here. This records each event of violence be it state, nonstate, or onesided. First thing we need to do is make it a spatial object.

In [ ]:
ged_geometry = [Point(xy) for xy in zip(ged_2014.longitude, ged_2014.latitude)]

In [ ]:
ged_gdf = gpd.GeoDataFrame(ged_2014, geometry = ged_geometry)

In [ ]:
ged_gdf.plot()

Now lets pick a few countries that we are interested in and make a new geospatial shapefile. 

In [ ]:
world_pts.CNTRY_NAME

In [ ]:
countrys = ['Uganda', 'Central African Republic', 'Congo, DRC']

In [ ]:
central_af = world_pts[world_pts.CNTRY_NAME.isin(countrys)]

In [ ]:
from fiona.crs import from_epsg
import matplotlib.pyplot as plt
central_af.crs = from_epsg(4326)
ged_gdf.crs = from_epsg(4326)

In [ ]:
fig, ax = plt.subplots()
central_af.plot(ax=ax, facecolor='gray');
ged_gdf.plot(ax=ax, color='red');
ax.set_aspect('equal')

In [ ]:
central_af.head()

In [ ]:
car_pts = ged_gdf.within(central_af.loc[64, 'geometry'])
car_data = ged_gdf.loc[car_pts]

In [ ]:
drc_pts = ged_gdf.within(central_af.loc[156, 'geometry'])
drc_data = ged_gdf.loc[drc_pts]

In [ ]:
uga_pts = ged_gdf.within(central_af.loc[188, 'geometry'])
uga_data = ged_gdf.loc[uga_pts]

In [ ]:
fig, ax = plt.subplots()
central_af.plot(ax=ax, facecolor='gray');
car_data.plot(ax=ax, color='blue', markersize=uga_data['best']);
drc_data.plot(ax=ax, color='red', markersize=uga_data['best']);
uga_data.plot(ax=ax, color='green', markersize=uga_data['best']);
ax.set_aspect('equal')

In [ ]:
deaths = [sum(car_data.best),sum(drc_data.best),sum(uga_data.best)]

In [ ]:
deaths

In [ ]:
central_af['deaths'] = deaths

In [ ]:
central_af

In [ ]:
central_af.plot(column='deaths', legend=True)

## BONUS Download Country Code Maker!

https://github.com/JELambert/ccode_replace_python

In [ ]:
from ccode_replace_python import replace_ccode_country as rcc
#ccode_df = rcc.ccode_make(unmade_df, 'Country Name')